## Examples of using of OpenAI's GPT through the API

#### Writer
    Kunwoong Kim

#### Created / Last edited
    2023.06.21 / 2023.06.21

#### Notes
    Please be aware of the billing associated with using OpenAI's API!
    This notebook is written based on:

<https://www.indiehackers.com/post/how-to-fine-tune-a-gpt-3-model-using-python-with-your-own-data-for-improved-performance-198dfe51d6>

<https://www.articulatepython.com/blog/finetune-openai-models>

<https://www.datacamp.com/tutorial/fine-tuning-gpt-3-using-the-open-ai-api-and-python>

<https://passwd.tistory.com/entry/Python-OpenAI-API-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0>

In [1]:
import openai
import os

In [2]:
"""""""""""""""""""""
OpenAI API: create your API key
"""""""""""""""""""""
# API key
openai.api_key = 'YOUR-API-KEY' # for sharing

In [3]:
"""""""""""""""""""""
ChatGPT usage examples
"""""""""""""""""""""

""" [1] Chat completion """
example_messages = [
    {"role": "system", "content": "You are a kind assistant."},
    {"role": "user", "content": "Who won the Korean Series in 1999?"},
    {"role": "assistant", "content": "Hanhwa Eagles won the Korean Series in 1999."},
    {"role": "user", "content": "Where was it played?"}
    ]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=example_messages
    )
response = completion.choices[0].message.content
print(f'The last prompt: {example_messages[-1]["content"]}')
print(f'Response: {response}')

""" [2] DIY """
# TODO: Do it yourself

The last prompt: Where was it played?
Response: The Korean Series in 1999 was played at Mokdong Baseball Stadium in Seoul, South Korea.


' [2] DIY '

In [ ]:
"""""""""""""""""""""
ChatGPT fine-tuning
"""""""""""""""""""""

""" Configuration """
# model engine
model = 'davinci'
# model hyperparams
n_epochs = 2
batch_size = 4
learning_rate_multiplier = 0.3
# Custom data
current_path = os.getcwd()
training_file = os.path.abspath(os.path.join(current_path, 'data/training_data.jsonl'))
validation_file = os.path.abspath(os.path.join(current_path, 'data/validation_data.jsonl'))
assert os.path.exists(training_file) and os.path.exists(validation_file)
# Upload to OpenAI
train_uploader = openai.File.create(file=open(training_file, 'rb'), purpose='fine-tune')
val_uploader = openai.File.create(file=open(validation_file, 'rb'), purpose='fine-tune')

# config
configs = {
    'model': model,
    'n_epochs': n_epochs,
    'batch_size': batch_size,
    'learning_rate_multiplier': learning_rate_multiplier,
    'training_file': train_uploader.id,
    'validation_file': val_uploader.id
}


""" Fine-tuning process """
# creation
fine_tuner = openai.FineTune.create(**configs)
fine_tuning_end = False
fine_tuning_events = []
while True:
    late_event = openai.FineTune.retrieve(id=fine_tuner.id)['events'][-1]['message'] # OR: openai.FineTune.list_events(id=fine_tuner.id)
    late_status = openai.FineTune.retrieve(id=fine_tuner.id)['status']
    fine_tuning_end = (late_status == 'succeeded') or (late_status == 'failed')
    if late_event not in fine_tuning_events:
        fine_tuning_events.append(late_event)
        print(f'[Info] current event: {late_event}')
    if fine_tuning_end:
        break
    
""" Inference"""
fine_tuned_model = openai.FineTune.retrieve(id=fine_tuner.id).fine_tuned_model
example_prompt = 'What is the capital of South Korea?->'
example_response = openai.Completion.create(
    model=fine_tuned_model,
    prompt=example_prompt,
    max_tokens=100,
    temperature=0.5,
    )
# inference with example prompt
print(f'Example response of {example_prompt} is \n \t {example_response["choices"][0]["text"]}')